In [4]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import sorting
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    sorting,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(edgeitems=5, precision=3, linewidth=300)

In [45]:
def crazy_chemist_mix(mixes: list, explosive: list) -> list:
    """Mix compounds from the `mixes` list. Avoid mixing `explosive` combinations.

    Mixed ingredients build a joint set. Adding an ingredient to another,
    adds it to the mix that the ingredient is already in. Ultimately,
    this is a disjoint set problem resulting in sets that don't combine
    any ingredients from the `explosive` list.

    Parameters
    ----------
    mixes : list
        pairs of ingredients to mix
    explosive : list
        pairs of ingredients to avoid

    Returns
    -------
    list
        a list of 0s and 1s, indicating when it is safe to mix ingredients
        for each entry in the `mixes` list.

    """
    n = max(max(a, b) for a, b in chain(mixes, explosive)) + 1
    parents = list(range(n))
    size = [1] * n

    def parent(x: int) -> int:
        """Find the parent of `x`. Compress the path."""
        while x != parents[x]:
            # Compress the set by halving.
            x, parents[x] = parents[x], parents[parents[x]]
        return x

    emap = [set() for _ in range(n)]
    for x, y in explosive:
        emap[x].add(y)
        emap[y].add(x)
    safety = []
    for a, b in mixes:
        a, b = parent(a), parent(b)  # noqa: PLW2901
        if a == b:
            safety.append(1)
            continue
        if a in emap[b] or b in emap[a]:
            # mixing a and b would produce an explosive mix
            safety.append(0)
            continue
        safety.append(1)
        if size[a] < size[b]:
            a, b = b, a  # noqa: PLW2901
        parents[b] = a
        size[a] += size[b]
        if emap[a] or emap[b]:
            emap[a] = emap[b] = set(map(parent, emap[a] | emap[b]))
    return safety

In [46]:
crazy_chemist_mix([[1, 2], [2, 3], [4, 5], [3, 5], [2, 4]], [[1, 3], [4, 2]])

[1, 0, 1, 1, 0]

In [47]:
crazy_chemist_mix([[1, 2], [2, 3], [1, 3]], [[1, 2], [1, 3]])

[0, 1, 0]

In [48]:
crazy_chemist_mix(
    [
        [11, 8],
        [16, 3],
        [1, 10],
        [13, 7],
        [20, 1],
        [11, 1],
        [11, 9],
        [20, 10],
        [12, 17],
        [20, 2],
        [17, 7],
        [15, 3],
        [14, 18],
        [11, 17],
        [2, 1],
        [10, 16],
        [5, 15],
        [15, 3],
        [14, 13],
        [11, 9],
    ],
    [
        [2, 15],
        [9, 16],
        [17, 8],
        [10, 9],
        [13, 5],
        [1, 18],
        [12, 19],
        [18, 11],
        [14, 19],
        [3, 1],
        [19, 17],
        [8, 5],
        [20, 1],
        [10, 6],
        [8, 16],
        [11, 2],
        [16, 10],
        [1, 8],
        [2, 12],
        [12, 9],
    ],
)

[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1]